# Laboratório 1 – Projeto Buscas

## Tarefa 2.1 - Grid Search

**Instituto Tecnológico de Aeronáutica – ITA**

**Inteligência Artificial – CMC-15**

**Professores:** 

Paulo André Castro

Paulo Marcelo Tasinaffo

**Alunos:**

Álvaro Tedeschi Neto

Davi Muniz de Vasconcelos

Gabriel Henrique Gobi

Thiago Lopes de Araujo

In [1]:
import numpy as np
import heapq
from math import inf, sqrt

In [3]:
class CostMap(object):
    """
    Represents a cost map where negative values indicates terrain which are no possible to transverse.
    """
    def __init__(self, map):
        """
        Creates a cost map.

        :param map: matrix with the obstacles.
        :type map: NumPy 2D array.
        """
        self.width = np.size(map, 1)
        self.height = np.size(map, 0)
        self.grid = np.ones((self.height, self.width))
        for i in range(self.height):
          for j in range(self.width):
            if map[i, j] != 0:
              self.add_obstacle((i, j), -1)  

    def get_cell_cost(self, i, j):
        """
        Obtains the cost of a cell in the cost map.

        :param i: the row (y coordinate) of the cell.
        :type i: int.
        :param j: the column (x coordinate) of the cell.
        :type j: int.
        :return: cost of the cell.
        :rtype: float.
        """
        return self.grid[i, j]

    def get_edge_cost(self, start, end):
        """
        Obtains the cost of an edge.

        :param start: tbe cell where the edge starts.
        :type start: float.
        :param end: the cell where the edge ends.
        :type end: float.
        :return: cost of the edge.
        :rtype: float.
        """
  
        return (self.get_cell_cost(start[0], start[1]) + self.get_cell_cost(end[0], end[1])) / 2.0

    def is_occupied(self, i, j):
        """
        Checks if a cell is occupied.

        :param i: the row of the cell.
        :type i: int.
        :param j: the column of the cell.
        :type j: int.
        :return: True if the cell is occupied, False otherwise.
        :rtype: bool.
        """
        return self.grid[i][j] < 0.0

    def is_index_valid(self, i, j):
        """
        Check if a (i,j) position is valid (is within the map boundaries).

        :param i: the row of the cell.
        :param i: int.
        :param j: the column of the cell.
        :param j: int.
        :return: if the index is valid.
        :rtype: bool.
        """
        return 0 <= i < self.height and 0 <= j < self.width

    def add_obstacle(self, cell, value):
        """
        Adds an obstacle given a cell region (x, y).

        :param cell: a grid cell defined as (x, y), where (0, 0) is the top left corner.
        :type cell: 2-dimensional tuple.
        :param value: the value used in the rectangular region.
        :type value: int.
        """
        i, j = cell
        if self.is_index_valid(i, j) and not self.is_occupied(i, j):
            self.grid[i, j] = value

In [4]:
map = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
                [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1],
                [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
                [1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
                [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
                [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
                [0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0],
                [0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1],
                [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0],
                [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
                [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
                [0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0],
                [0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0]])

cost_map = CostMap(map)

In [5]:
class Node(object):
    """
    Represents a node of a graph used for planning paths.
    """
    def __init__(self, i=0, j=0):
        """
        Creates a node of a graph used for planning paths.

        :param i: row of the node in the occupancy grid.
        :type i: int.
        :param j: column of the node in the occupancy grid.
        :type j: int.
        """
        self.i = i
        self.j = j
        self.f = inf
        self.g = inf
        self.closed = False
        self.parent = None

    def get_position(self):
        """
        Obtains the position of the node as a tuple.

        :return: (i, j) where i is the row and the column of the node, respectively.
        :rtype: 2-dimensional tuple of int.
        """
        return self.i, self.j

    def set_position(self, i, j):
        """
        Sets the position of this node.

        :param i: row of the node in the occupancy grid.
        :type i: int.
        :param j: column of the node in the occupancy grid.
        :type j: int.
        """
        self.i = i
        self.j = j

    def reset(self):
        """
        Resets the node to prepare it for a new path planning.
        """
        self.f = inf
        self.g = inf
        self.closed = False
        self.parent = None

    def distance_to(self, i, j):
        """
        Computes the distance from this node to the position (i, j).

        :param i: row of the target position.
        :type i: int.
        :param j: column of the target position.
        :type j: int.
        :return: distance from this node to (i, j).
        :rtype: float.
        """
        return sqrt((self.i - i) ** 2 + (self.j - j) ** 2)

    def __lt__(self, another_node):
        if self.i < another_node.i:
            return True
        if self.j < another_node.j:
            return True
        return False

class NodeGrid(object):
    """
    Represents a grid of graph nodes used by the planning algorithms.
    """
    def __init__(self, cost_map):
        """
        Creates a grid of graph nodes.

        :param cost_map: cost map used for planning.
        :type cost_map: CostMap.
        """
        self.cost_map = cost_map
        self.width = cost_map.width
        self.height = cost_map.height
        self.grid = np.empty((self.height, self.width), dtype=Node)
        for i in range(np.size(self.grid, 0)):
            for j in range(np.size(self.grid, 1)):
                self.grid[i, j] = Node(i, j)

    def reset(self):
        """
        Resets all nodes of the grid.
        """
        for row in self.grid:
            for node in row:
                node.reset()

    def get_node(self, i, j):
        """
        Obtains the node at row i and column j.

        :param i: row of the node.
        :type i: int.
        :param j: column of the node.
        :type j: int.
        :return: node at row i and column j.
        :rtype: Node.
        """
        return self.grid[i, j]

    def get_successors(self, i, j):
        """
        Obtains a list of the 8-connected successors of the node at (i, j).

        :param i: row of the node.
        :type i: int.
        :param j: column of the node.
        :type j: int.
        :return: list of the 8-connected successors.
        :rtype: list of Node.
        """
        successors = []
        for di, dj in [(-1, 0), (0, -1), (1, 0), (0, 1)]:
            if self.cost_map.is_index_valid(i + di, j + dj) and not self.cost_map.is_occupied(i + di, j + dj):
                successors.append((i + di, j + dj))
        return successors

In [6]:
class PathPlanner(object):
    """
    Represents a path planner, which may use Dijkstra, Greedy Search or A* to plan a path.
    """

    def __init__(self, cost_map):
        """
        Creates a new path planner for a given cost map.

        :param cost_map: cost used in this path planner.
        :type cost_map: CostMap.
        """
        self.cost_map = cost_map
        self.node_grid = NodeGrid(cost_map)

    @staticmethod
    def construct_path(goal_node):
        """
        Extracts the path after a planning was executed.

        :param goal_node: node of the grid where the goal was found.
        :type goal_node: Node.
        :return: the path as a sequence of (x, y) positions: [(x1,y1),(x2,y2),(x3,y3),...,(xn,yn)].
        :rtype: list of tuples.
        """
        node = goal_node
        # Since we are going from the goal node to the start node following the parents, we
        # are transversing the path in reverse
        reversed_path = []
        while node is not None:
            reversed_path.append(node.get_position())
            node = node.parent
        return reversed_path[::-1]  # This syntax creates the reverse list

    def a_star(self, start_position, goal_position):
        """
        Plans a path using A*.

        :param start_position: position where the planning stars as a tuple (x, y).
        :type start_position: tuple.
        :param goal_position: goal position of the planning as a tuple (x, y).
        :type goal_position: tuple.
        :return: the path as a sequence of positions and the path cost.
        :rtype: list of tuples and float.
        """
        # The first return is the path as sequence of tuples (as returned by the method construct_path())
        # The second return is the cost of the path

        self.node_grid.reset()  # resets the grid
        pq = []  # initialize a list to be our priority queue

        start = self.node_grid.get_node(*start_position)  # start Node
        goal = self.node_grid.get_node(*goal_position)  # goal Node

        start.g = 0
        start.f = start.distance_to(*goal_position)

        heapq.heappush(pq, (start.f, start))

        while pq:
            cost, node = heapq.heappop(pq)
            if node == goal:
                return self.construct_path(goal), node.g
            node.closed = True
            node_position = node.get_position()
            for successor_position in self.node_grid.get_successors(
                    *node_position):
                successor = self.node_grid.get_node(*successor_position)
                if not successor.closed:
                    if successor.f > node.g + self.cost_map.get_edge_cost(
                     node_position,successor_position) + successor.distance_to(*goal_position):
                        successor.g = node.g + self.cost_map.get_edge_cost(
                            node_position, successor_position)
                        successor.f = successor.g + successor.distance_to(*goal_position)
                        successor.parent = node
                        heapq.heappush(pq, (successor.f, successor))
        return [], inf

In [7]:
# Create the path planner using the cost map
path_planner = PathPlanner(cost_map)

start_position = (0, 0)
goal_position = (20, 23)
path, cost = path_planner.a_star(start_position, goal_position)
print(cost)

43.0
